## Types 🧱

The most basic agentic building block in Agentics is the **TYPE**.

A **TYPE** is a typed state schema, implemented as a Pydantic `BaseModel`, that describes **what a piece of information looks like** in your workflow. Instead of passing around loose dicts or ad-hoc JSON, you work with well-defined ATypes that:

- Specify **fields**, **types**, and **descriptions** (e.g. `name: str`, `age: int | None`).
- Act as the **input and output types** of transducible functions.
- Provide a **stable contract** between different stages of your pipeline (ingestion → enrichment → classification → summarization, etc.).

You can think of an Type as the “vocabulary word” for your agentic system:

- `UserMessage`, `EmailDraft`, `ProductReview`, `Movie`, `Summary`, `SQLQuery`, ...
- Each one captures a coherent **state of the world** at some step of reasoning.

Because ATypes are just Pydantic models, you automatically get:

- Validation and normalization of incoming data.
- Rich metadata (field descriptions) that guide LLM behavior.
- Compatibility with Map–Reduce, memory, and transducible composition.

In practice, you’ll spend much of your time in Agentics **defining, refining, and composing ATypes**, and then wiring them together with transducible functions to form robust, structured reasoning pipelines.

In [ ]:
from typing import Optional, List
from pydantic import BaseModel, Field

class CourseEnrollment(BaseModel):
    student_name: Optional[str] = Field(
        None,
        description="Full name of the student.",
    )
    student_age: Optional[int] = Field(
        None,
        description="Age of the student in years.",
    )
    enrolled_courses: Optional[List[str]] = Field(
        None,
        description="List of course names the student is enrolled in.",
    )

## Operations between Types ⚙️

Agentics treats ATypes as **algebraic objects** you can combine to build richer schemas and pipelines.  
At the **type level**, two primitive operations are:

- **Type Merge**: `X & Y`  
- **Type Composition**: `Z @ Y`  

These do **not** run any LLMs by themselves – they define new *types* that you can later use in transductions.

---

### 1. Type Merge `X & Y` – Combine Fields

**Merge** takes two ATypes and creates a new AType whose fields are the **union** of both.  
If fields share the same name, the merge follows your implementation rules (e.g., right-hand side wins, or enforced compatibility).

Conceptually:

> `Merged = X & Y`  
> “A type that has all fields of `X` and all fields of `Y`.”

In [ ]:
from typing import Optional, List
from pydantic import BaseModel, Field

class Movie(BaseModel):
    movie_name: Optional[str] = Field(None, description="Movie title.")
    description: Optional[str] = Field(None, description="Short plot summary.")
    year: Optional[int] = Field(None, description="Year of release.")

class Genre(BaseModel):
    genre: Optional[str] = Field(
        None,
        description="Single genre label, e.g. 'drama', 'comedy', 'thriller'.",
    )

# Type-level merge: all fields from Movie and Genre
MovieWithGenre = Movie & Genre
print(MovieWithGenre.model_fields)

### Type Composition Z @ Y 

Composition expresses pipeline structure between ATypes:

Z @ Y

“A type-level composition where Y feeds into Z in the logical pipeline.”

Think of it as describing:

“First I obtain something of type Y, then from that I obtain something of type Z.”

You still need transducible functions to execute this, but @ gives you a compact way to name and reason about multi-step flows.


In [ ]:
class Email(BaseModel):
    to: Optional[str] = Field(None, description="Recipient name or email.")
    subject: Optional[str] = Field(None, description="Email subject line.")
    body: Optional[str] = Field(None, description="Email body text.")

class Summary(BaseModel):
    summary_text: Optional[str] = Field(
        None,
        description="Concise summary of the source content.",
    )

# Type-level composition: “summarize an email”
EmailSummary = Summary @ Email
print(EmailSummary.model_fields)

### Merge + Composition Together

A common pattern is:
- Merge to enrich a state with additional attributes.
- Compose to describe what happens next in the pipeline.

The example below tells you, purely at the type level:
- EmailWithSentiment contains both the raw email and its sentiment.
- SummarizedEmailWithSentiment is “a summary of an email that has already been sentiment-annotated.”


In [ ]:
class Sentiment(BaseModel):
    sentiment: Optional[str] = Field(
        None,
        description="Overall sentiment, e.g. 'positive', 'neutral', 'negative'.",
    )

# 1. Merge: email content + sentiment label
EmailWithSentiment = Email & Sentiment

# 2. Composition: summarize the enriched email
SummarizedEmailWithSentiment = Summary @ EmailWithSentiment
print(SummarizedEmailWithSentiment.model_fields)

### Synthetic Data Generation 🧪

Agentics provides a helper, `generate_prototypical_instances`, that asks the LLM to create
**representative examples** of a given `BaseModel` type.

You pass:

- a Pydantic class (your AType), and  
- the number of instances you want (`n_instances`)  

and it returns a list of fully-typed objects that respect:

- your field names  
- their Python types  
- and their `Field(..., description="...")` metadata (used as guidance for the model)

This is useful for:

- Quickly generating **sample data** for demos and tests  
- Building **few-shot examples** consistent with your schema  
- Exploring how a newly designed Type “behaves” in practice

In [ ]:
from agentics.core.transducible_functions import generate_prototypical_instances

products = await generate_prototypical_instances(CourseEnrollment, n_instances=5)
for product in products:
    print(product.model_dump_json(indent=2))


### Dynamic Type Generation ✨

Agentics enables **on-the-fly generation of Pydantic types from natural language descriptions**.  
Instead of hand-coding every schema up front, you can describe a structure in plain English and let a transducible function synthesize a valid Pydantic model for you.

A typical dynamic type generation pipeline looks like this:

1. You provide a natural-language description of the type you want.
2. A transducible function (e.g. `generate_atype_from_description`) uses an LLM to produce:
   - Python code for a Pydantic `BaseModel` (fields, types, and descriptions).
   - A concrete `BaseModel` subclass imported from that code.
3. The resulting type can be used like any other Pydantic model in Agentics:
   - as an `atype` for `AG`,
   - as `InputModel` / `OutputModel` for transducible functions,
   - or as an intermediate typed state in a larger pipeline.

#### Example: Generate a `Person` Type from Natural Language

In [ ]:
from agentics.core.transducible_functions import generate_atype_from_description
person = await generate_atype_from_description("Person with demographic information including name, age, email and birthplace")
citizenship = await generate_atype_from_description("Citizenship information including country of citizenship and visa status")
print(person.model_dump_json(indent=2))
print(citizenship.model_dump_json(indent=2))

## Logical Proximity (LP) 🧭

**Logical Proximity (LP)** measures *how naturally* information can flow from one Pydantic type to another within a transduction.  
It gives you a numeric intuition for:

> “How well does type `X` contain the information needed to become type `Y`?”

Formally, we write this as:

> `LP(X → Y) ∈ [0, 1]`

where:

- `LP(X → Y) ≈ 1` means:  
  > “`X` has almost everything needed to produce `Y` in a stable, evidence-based way.”
- `LP(X → Y) ≈ 0` means:  
  > “`X` and `Y` are logically unrelated; no meaningful transduction is expected.”

In [ ]:
from agentics.core.transducible_functions import estimateLogicalProximity
print(await estimateLogicalProximity(citizenship.atype << person.atype))


### Logical Proximity - Key Properties

1. **Asymmetry (Not Symmetric)**  
   Logical proximity is **directional**:

   - `LP(X → Y)` is *not* necessarily equal to `LP(Y → X)`.
   - Example: a detailed `Movie` object can naturally give rise to a `Genre`, but the reverse is not true:
     - `LP(Movie → Genre)` might be high (e.g. 0.9).
     - `LP(Genre → Movie)` should be low (e.g. 0.1), because a genre alone cannot reconstruct the full movie.

2. **Zero Proximity for Unrelated Types**  
   If two types share no meaningful overlap in:

   - field semantics (names + descriptions),
   - data types,
   - or real-world concept,

   then:

   > `LP(X → Y) = 0`

   This encodes: “There is no evidence-preserving way to transduce `X` into `Y`.”